# LangGraph and LangSmith - Agentic RAG Powered by LangChain

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating our Tool Belt
  4. Creating Our State
  5. Creating and Compiling A Graph!

- 🤝 Breakout Room #2:
  1. Evaluating the LangGraph Application with LangSmith
  2. Adding Helpfulness Check and "Loop" Limits
  3. LangGraph for the "Patterns" of GenAI

# 🤝 Breakout Room #1

## Part 1: LangGraph - Building Cyclic Applications with LangChain

LangGraph is a tool that leverages LangChain Expression Language to build coordinated multi-actor and stateful applications that includes cyclic behaviour.

### Why Cycles?

In essence, we can think of a cycle in our graph as a more robust and customizable loop. It allows us to keep our application agent-forward while still giving the powerful functionality of traditional loops.

Due to the inclusion of cycles over loops, we can also compose rather complex flows through our graph in a much more readable and natural fashion. Effectively allowing us to recreate application flowcharts in code in an almost 1-to-1 fashion.

### Why LangGraph?

Beyond the agent-forward approach - we can easily compose and combine traditional "DAG" (directed acyclic graph) chains with powerful cyclic behaviour due to the tight integration with LCEL. This means it's a natural extension to LangChain's core offerings!

## Task 1:  Dependencies


## Task 2: Environment Variables

We'll want to set both our OpenAI API key and our LangSmith environment variables.

In [45]:
import os
# import getpass
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")


# os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [ ]:
# os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY")

In [46]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE7 - LangGraph - {uuid4().hex[0:8]}"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

## Task 3: Creating our Tool Belt

As is usually the case, we'll want to equip our agent with a toolbelt to help answer questions and add external knowledge.

There's a tonne of tools in the [LangChain Community Repo](https://github.com/langchain-ai/langchain-community/tree/main/libs/community) but we'll stick to a couple just so we can observe the cyclic nature of LangGraph in action!

We'll leverage:

- [Tavily Search Results](https://github.com/langchain-ai/langchain-community/blob/main/libs/community/langchain_community/tools/tavily_search/tool.py)
- [Arxiv](https://github.com/langchain-ai/langchain-community/blob/main/libs/community/langchain_community/tools/arxiv/tool.py)

#### 🏗️ Activity #1:

Please add the tools to use into our toolbelt.

> NOTE: Each tool in our toolbelt should be a method.

In [47]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.arxiv.tool import ArxivQueryRun

# Added 2 new tools
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.agents import tool

tavily_tool = TavilySearchResults(max_results=5)
wiki_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

@tool
def evaluate_expression(expr: str) -> float:
    """Evaluate a simple math expression like '3 * (4 + 2)'."""
    try:
        return eval(expr)
    except Exception as e:
        return f"Error: {str(e)}"

tool_belt = [
    tavily_tool,
    ArxivQueryRun(),
    wiki_tool,
    evaluate_expression
]

### Model

Now we can set-up our model! We'll leverage the familiar OpenAI model suite for this example - but it's not *necessary* to use with LangGraph. LangGraph supports all models - though you might not find success with smaller models - as such, they recommend you stick with:

- OpenAI's GPT-3.5 and GPT-4
- Anthropic's Claude
- Google's Gemini

> NOTE: Because we're leveraging the OpenAI function calling API - we'll need to use OpenAI *for this specific example* (or any other service that exposes an OpenAI-style function calling API.

In [48]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

Now that we have our model set-up, let's "put on the tool belt", which is to say: We'll bind our LangChain formatted tools to the model in an OpenAI function calling format.

In [49]:
model = model.bind_tools(tool_belt)

#### ❓ Question #1:
How does the model determine which tool to use?

Answer:

The model determines which tool to use through OpenAI's function calling mechanism combined with tool descriptions and the current context. Here's how it works:

1. Model reads the question
2. Looks at available tool descriptions
3. Picks the tool that best matches what we are asking for
4. If no tool fits, just answers normally

Basically model just picks the right tool based on user query!


## Task 4: Putting the State in Stateful

Earlier we used this phrasing:

`coordinated multi-actor and stateful applications`

So what does that "stateful" mean?

To put it simply - we want to have some kind of object which we can pass around our application that holds information about what the current situation (state) is. Since our system will be constructed of many parts moving in a coordinated fashion - we want to be able to ensure we have some commonly understood idea of that state.

LangGraph leverages a `StatefulGraph` which uses an `AgentState` object to pass information between the various nodes of the graph.

There are more options than what we'll see below - but this `AgentState` object is one that is stored in a `TypedDict` with the key `messages` and the value is a `Sequence` of `BaseMessages` that will be appended to whenever the state changes.

Let's think about a simple example to help understand exactly what this means (we'll simplify a great deal to try and clearly communicate what state is doing):

1. We initialize our state object:
  - `{"messages" : []}`
2. Our user submits a query to our application.
  - New State: `HumanMessage(#1)`
  - `{"messages" : [HumanMessage(#1)}`
3. We pass our state object to an Agent node which is able to read the current state. It will use the last `HumanMessage` as input. It gets some kind of output which it will add to the state.
  - New State: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
  - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. We pass our state object to a "conditional node" (more on this later) which reads the last state to determine if we need to use a tool - which it can determine properly because of our provided object!

In [50]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

## Task 5: It's Graphing Time!

Now that we have state, and we have tools, and we have an LLM - we can finally start making our graph!

Let's take a second to refresh ourselves about what a graph is in this context.

Graphs, also called networks in some circles, are a collection of connected objects.

The objects in question are typically called nodes, or vertices, and the connections are called edges.

Let's look at a simple graph.

![image](https://i.imgur.com/2NFLnIc.png)

Here, we're using the coloured circles to represent the nodes and the yellow lines to represent the edges. In this case, we're looking at a fully connected graph - where each node is connected by an edge to each other node.

If we were to think about nodes in the context of LangGraph - we would think of a function, or an LCEL runnable.

If we were to think about edges in the context of LangGraph - we might think of them as "paths to take" or "where to pass our state object next".

Let's create some nodes and expand on our diagram.

> NOTE: Due to the tight integration with LCEL - we can comfortably create our nodes in an async fashion!

In [51]:
from langgraph.prebuilt import ToolNode

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

tool_node = ToolNode(tool_belt)

Now we have two total nodes. We have:

- `call_model` is a node that will...well...call the model
- `tool_node` is a node which can call a tool

Let's start adding nodes! We'll update our diagram along the way to keep track of what this looks like!


In [52]:
from langgraph.graph import StateGraph, END

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)

Let's look at what we have so far:

![image](https://i.imgur.com/md7inqG.png)

Next, we'll add our entrypoint. All our entrypoint does is indicate which node is called first.

In [53]:
uncompiled_graph.set_entry_point("agent")

![image](https://i.imgur.com/wNixpJe.png)

Now we want to build a "conditional edge" which will use the output state of a node to determine which path to follow.

We can help conceptualize this by thinking of our conditional edge as a conditional in a flowchart!

Notice how our function simply checks if there is a "function_call" kwarg present.

Then we create an edge where the origin node is our agent node and our destination node is *either* the action node or the END (finish the graph).

It's important to highlight that the dictionary passed in as the third parameter (the mapping) should be created with the possible outputs of our conditional function in mind. In this case `should_continue` outputs either `"end"` or `"continue"` which are subsequently mapped to the action node or the END node.

In [54]:
def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
    "agent",
    should_continue
)

Let's visualize what this looks like.

![image](https://i.imgur.com/8ZNwKI5.png)

Finally, we can add our last edge which will connect our action node to our agent node. This is because we *always* want our action node (which is used to call our tools) to return its output to our agent!

In [55]:
uncompiled_graph.add_edge("action", "agent")

Let's look at the final visualization.

![image](https://i.imgur.com/NWO7usO.png)

All that's left to do now is to compile our workflow - and we're off!

In [56]:
simple_agent_graph = uncompiled_graph.compile()

#### ❓ Question #2:

Is there any specific limit to how many times we can cycle?

Answer:

Yes, LangGraph enforces a default recursion limit of 25. That means an agent can cycle between nodes—like "agent" and "action"—up to 25 times before the graph halts automatically. This built-in safeguard helps prevent infinite loops, runaway API calls, and excessive costs.


The standard way to impose a limit is by using the recursion_limit parameter when compiling the graph. For example:

```python
graph = uncompiled_graph.compile(recursion_limit=5)
```

This sets a hard cap—once the agent hits 5 recursive steps, the graph will stop automatically. It’s the recommended and built-in way to prevent runaway loops.

Apart from that, we can also add custom limits inside our node logic. One common method is checking how many messages have been exchanged:

```python
if len(state["messages"]) > 10:
    return END
```

This is helpful in chat scenarios where we want to stop after a certain number of back-and-forths.
Another way is to look at the content of the message and stop based on a condition:

```python
if "STOP" in state["latest_message"]["content"]:
    return END
```

This gives more flexibility and lets the agent exit based on dynamic inputs.

In short, LangGraph doesn’t enforce a limit by default, but recursion_limit is the standard way to do it, and we can always layer on additional logic if needed.


## Using Our Graph

Now that we've created and compiled our graph - we can call it *just as we'd call any other* `Runnable`!

Let's try out a few examples to see how it fairs:

In [57]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="Who is the current captain of the Winnipeg Jets?")]}

async for chunk in simple_agent_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_FlzrbIUaDKtUksSf5JVdoWIB', 'function': {'arguments': '{"query":"current captain of the Winnipeg Jets"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 250, 'total_tokens': 273, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': None, 'id': 'chatcmpl-BtC6BraVBGeYWU5rql2oJX92JokCW', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--c4a9d22a-abec-40ee-8205-032685f36d7a-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current captain of the Winnipeg Jets'}, 'id': 'call_FlzrbIUaDKtUksSf5JVdoWIB', 'type': 

Let's look at what happened:

1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "tool_calls" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "tool_calls" `additional_kwarg` and passed the state object to END where we see it output in the cell above!

Now let's look at an example that shows a multiple tool usage - all with the same flow!

In [58]:
inputs = {"messages" : [HumanMessage(content="Search Arxiv for the QLoRA paper, then search each of the authors to find out their latest Tweet using Tavily!")]}

async for chunk in simple_agent_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
          print(f"Tool Used: {values['messages'][0].name}")
        print(values["messages"])

        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_aKNJVbV2s4asVbewUznH6EOv', 'function': {'arguments': '{"query": "QLoRA"}', 'name': 'arxiv'}, 'type': 'function'}, {'id': 'call_ezTV00rCpgaERmxRlGqwewmr', 'function': {'arguments': '{"query": "latest Tweet of the author of QLoRA"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 266, 'total_tokens': 324, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': None, 'id': 'chatcmpl-BtC6LUdqujOfeq3wBBtgVl2q5n8ri', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--9af09cae-253b-4ba1-a874-2d1a9395d32f-0', tool_calls=[{'name': 'arxiv', 

In [59]:
# 🔢 Test: Math tool
math_test = {"messages": [HumanMessage(content="What is (5 + 3) * 2?")]}

async for chunk in simple_agent_graph.astream(math_test, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Math tool - from node: {node}")
        print(values["messages"])
        print("\n")

# 📚 Test: Wikipedia tool
wiki_test = {"messages": [HumanMessage(content="Give a short summary about LangChain from Wikipedia.")]}

async for chunk in simple_agent_graph.astream(wiki_test, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Wiki tool - from node: {node}")
        print(values["messages"])
        print("\n")


Math tool - from node: agent
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_P2aSibYe3tmY9v9af1MUrV2Z', 'function': {'arguments': '{"expr":"(5 + 3) * 2"}', 'name': 'evaluate_expression'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 252, 'total_tokens': 274, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': None, 'id': 'chatcmpl-BtC6W0a4cxI7WhirI4XBW2ZZEr4aN', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ddd8d3be-40df-4c10-8fdf-2315cc6c68da-0', tool_calls=[{'name': 'evaluate_expression', 'args': {'expr': '(5 + 3) * 2'}, 'id': 'call_P2aSibYe3tmY9v9af1MUrV2Z', 'type': 'tool_call'}], usage_metadata={'input_tokens': 252, 'output_tokens': 22, 

#### 🏗️ Activity #2:

Please write out the steps the agent took to arrive at the correct answer.

Answer:

Based on the test output, here are the steps the agent took to arrive at the correct answer for "What is (5 + 3) * 2?":

1. **Initial Processing (Agent Node)**: The agent received the user's math question and determined it needed to use a tool to calculate the result. It generated a tool call for the `evaluate_expression` function with the argument `"(5 + 3) * 2"`.

2. **Tool Execution (Action Node)**: The system executed the `evaluate_expression` tool, which evaluated the mathematical expression and returned the result `16`.

3. **Final Response (Agent Node)**: The agent received the tool result and formulated a natural language response: "The result of (5 + 3) * 2 is 16."

4. **Completion**: Since there were no more tool calls needed and the response was complete, the conditional edge directed the flow to END, finishing the conversation.

**Summary of the flow**: User Input → Agent (tool call) → Action (execute math tool) → Agent (final response) → END


# 🤝 Breakout Room #2

## Part 1: LangSmith Evaluator

### Pre-processing for LangSmith

To do a little bit more preprocessing, let's wrap our LangGraph agent in a simple chain.

In [60]:
def convert_inputs(input_object):
  return {"messages" : [HumanMessage(content=input_object["question"])]}

def parse_output(input_state):
  return input_state["messages"][-1].content

agent_chain_with_formatting = convert_inputs | simple_agent_graph | parse_output

In [63]:
agent_chain_with_formatting.invoke({"question" : "What is RAG?"})
agent_chain_with_formatting.invoke({
    "question": "Summarize the Wikipedia entry for OpenAI."
})
agent_chain_with_formatting.invoke({
    "question": "What is (5 + 3) * 2?"
})


/Users/mayankshah/Documents/ai-makerspace/aie7-bootcamp/AIE7_local/05_Our_First_Agent_with_LangGraph/.venv/lib/python3.13/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/mayankshah/Documents/ai-makerspace/aie7-bootcamp/AIE7_local/05_Our_First_Agent_with_LangGraph/.venv/lib/python3.13/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


'The result of (5 + 3) * 2 is 16.'

### Task 1: Creating An Evaluation Dataset

Just as we saw last week, we'll want to create a dataset to test our Agent's ability to answer questions.

In order to do this - we'll want to provide some questions and some answers. Let's look at how we can create such a dataset below.

```python
questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?",
    "What is the result of 3 * (7 + 1)?",
    "Summarize the Wikipedia entry for OpenAI."        
]

answers = [
    {"must_mention" : ["paged", "optimizer"]},
    {"must_mention" : ["NF4", "NormalFloat"]},
    {"must_mention" : ["ground", "context"]},
    {"must_mention" : ["Tim", "Dettmers"]},
    {"must_mention" : ["PyTorch", "TensorFlow"]},
    {"must_mention" : ["reduce", "parameters"]},
    {"must_mention": ["24"]}, 
    {"must_mention": ["OpenAI", "research"]} 
]
```

#### 🏗️ Activity #3:

Please create a dataset in the above format with at least 5 questions.

In [25]:
questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?",
    "What is the result of 3 * (7 + 1)?",
    "Summarize the Wikipedia entry for OpenAI."  
]

answers = [
    {"must_mention" : ["paged", "optimizer"]},
    {"must_mention" : ["NF4", "NormalFloat"]},
    {"must_mention" : ["ground", "context"]},
    {"must_mention" : ["Tim", "Dettmers"]},
    {"must_mention" : ["PyTorch", "TensorFlow"]},
    {"must_mention" : ["reduce", "parameters"]},
    {"must_mention": ["24"]}, 
    {"must_mention": ["OpenAI", "research"]} 
]

In [26]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="Who is the current captain of the Winnipeg Jets?")]}

async for chunk in simple_agent_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_vG8DVoAAYOWSTTQFHcl7VK7Z', 'function': {'arguments': '{"query":"current captain of the Winnipeg Jets"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 250, 'total_tokens': 273, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': None, 'id': 'chatcmpl-Bt3X3iW4j38IrpNBRtmXHE0rECMje', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--c8bae562-68e1-4c40-ba50-bb30bdc6f829-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current captain of the Winnipeg Jets'}, 'id': 'call_vG8DVoAAYOWSTTQFHcl7VK7Z', 'type': 

In [27]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="Who is the current captain of the Winnipeg Jets?")]}

async for chunk in simple_agent_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2xI0FHsfsNgkrKL9Gi2GZ20J', 'function': {'arguments': '{"query":"current captain of the Winnipeg Jets"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 250, 'total_tokens': 273, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': None, 'id': 'chatcmpl-Bt3XEAy9ssiE56aJibQRgxiDhbUpq', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--5fb0e230-afeb-49f3-a068-e7a2118cd21c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current captain of the Winnipeg Jets'}, 'id': 'call_2xI0FHsfsNgkrKL9Gi2GZ20J', 'type': 

Now we can add our dataset to our LangSmith project using the following code which we saw last Thursday!

In [28]:
from langsmith import Client

client = Client()

dataset_name = f"Retrieval Augmented Generation - Evaluation Dataset - {uuid4().hex[0:8]}"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions about the QLoRA Paper to Evaluate RAG over the same paper."
)

client.create_examples(
    inputs=[{"question" : q} for q in questions],
    outputs=answers,
    dataset_id=dataset.id,
)

{'example_ids': ['92d6e497-847a-42e2-a43f-97a6d963c402',
  'd0c486fa-98bf-4144-9396-e3733f4476dd',
  'b3266e73-82d0-447f-a8fb-6f9d68ad5f24',
  '7bda2f48-41b7-4117-a191-db6537578f50',
  'b5757379-c4e3-4d74-b6b2-9105c2da8fd2',
  '8282b70d-6f07-45ba-91a1-3416ddb5bdf6',
  '199b0465-f341-438a-a040-9dbf37c03089',
  '98aeaa2f-4041-4737-a84b-5dc679ee79ea'],
 'count': 8}

#### ❓ Question #3:

How are the correct answers associated with the questions?

> NOTE: Feel free to indicate if this is problematic or not

Answer:

The correct answers are associated with the questions through positional indexing - meaning the order of elements in both lists determines the correspondence. The first question (index 0) matches the first answer (index 0), the second question (index 1) matches the second answer (index 1), and so forth.

```python
questions = [
    "What optimizer is used in QLoRA?",                    # Position 0
    "What data type was created in the QLoRA paper?",      # Position 1
    "What is a Retrieval Augmented Generation system?",    # Position 2
    # ... etc
]

answers = [
    {"must_mention" : ["paged", "optimizer"]},      # Position 0 → matches question 0
    {"must_mention" : ["NF4", "NormalFloat"]},      # Position 1 → matches question 1  
    {"must_mention" : ["ground", "context"]},       # Position 2 → matches question 2
    # ... etc
]
```

In the above case the first question is mapped to the first answer. Similarly for all pairings in the list.

When the dataset is created the list is zipped together based on their positions.

The above approach is problematic because of several reasons.
1. Fragile Dependencies: Any modification to one list without corresponding changes to the other breaks all associations.
2. Silent Failures: Mismatched indices won't throw errors but will create incorrect question-answer pairings.
3. Maintenance Overhead: Developers must manually ensure both lists stay synchronized
4. Poor Readability: The relationship between questions and answers isn't immediately apparent
5. No Validation: No automatic checks to ensure lists have matching lengths or logical pairings

The more robust approach would be to use a list of dictionaries or tuples where each question is explicitly paired with its answer.



### Task 2: Adding Evaluators

Now we can add a custom evaluator to see if our responses contain the expected information.

We'll be using a fairly naive exact-match process to determine if our response contains specific strings.

In [29]:
from langsmith.evaluation import EvaluationResult, run_evaluator

@run_evaluator
def must_mention(run, example) -> EvaluationResult:
    prediction = run.outputs.get("output") or ""
    required = example.outputs.get("must_mention") or []
    score = all(phrase in prediction for phrase in required)
    return EvaluationResult(key="must_mention", score=score)

#### ❓ Question #4:

What are some ways you could improve this metric as-is?

> NOTE: Alternatively you can suggest where gaps exist in this method.

Answer:

There are certain limitations with the current metric:

1. Exact Match Only:
   It only looks for exact phrases, so it misses correct answers that use synonyms or paraphrasing.
2. No Context Check:
   It doesn’t verify if the required words are used meaningfully or just listed.
3. All-or-Nothing Scoring:
   If even one required phrase is missing, the answer gets no credit.
4. Insensitive to Case/Formatting:
   It may fail if the answer uses different capitalization or punctuation.

There are several ways to improve the current metric.
1. Fuzzy or Case-Insensitive Matching:
   Allow minor spelling differences and ignore capitalization.
2. Synonym/Semantic Matching:
   Use NLP tools to recognize similar meanings, not just exact words.
3. Partial Credit:
   Score based on how many required concepts are present, not just all-or-nothing.
4. Contextual Validation:
   Check that required phrases are used in a relevant and correct context, not just mentioned.



Task 3: Evaluating

All that is left to do is evaluate our agent's response!

In [30]:
experiment_results = client.evaluate(
    agent_chain_with_formatting,
    data=dataset_name,
    evaluators=[must_mention],
    experiment_prefix=f"Search Pipeline - Evaluation - {uuid4().hex[0:4]}",
    metadata={"version": "1.0.0"},
)

View the evaluation results for experiment: 'Search Pipeline - Evaluation - ddc9-7009ebe5' at:
https://smith.langchain.com/o/3c2c7006-57b9-4cbe-911e-6f73b4734883/datasets/93032b24-f940-4271-898f-1388b87db08a/compare?selectedSessions=8bca64de-8d0f-4ed1-b4b3-706f278a75c1




0it [00:00, ?it/s]

In [23]:
experiment_results

<ExperimentResults Search Pipeline - Evaluation - e10f-8b5d3d5a>

## Part 2: LangGraph with Helpfulness:

### Task 3: Adding Helpfulness Check and "Loop" Limits

Now that we've done evaluation - let's see if we can add an extra step where we review the content we've generated to confirm if it fully answers the user's query!

We're going to make a few key adjustments to account for this:

1. We're going to add an artificial limit on how many "loops" the agent can go through - this will help us to avoid the potential situation where we never exit the loop.
2. We'll add to our existing conditional edge to obtain the behaviour we desire.

First, let's define our state again - we can check the length of the state object, so we don't need additional state for this.

In [31]:
class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

Now we can set our graph up! This process will be almost entirely the same - with the inclusion of one additional node/conditional edge!

#### 🏗️ Activity #5:

Please write markdown for the following cells to explain what each is doing.



Answer:

In this cell, we begin constructing a new LangGraph called graph_with_helpfulness_check using the StateGraph API with the AgentState type. This graph will manage the flow of messages between the agent (LLM) and the tools.

We define two main nodes:

"agent" node:
This represents the thinking step of the agent. It uses the call_model function, which invokes the language model to process the user’s input and decide whether to respond directly or trigger a tool.

"action" node:
This node is responsible for executing any tool calls made by the agent. It uses the tool_node component, which wraps all available tools (Wikipedia, Math evaluator, Arxiv, Tavily) and ensures the correct one is called based on the model’s output.

This structure sets up the core building blocks for a reasoning-and-action loop: the agent can think, decide to use a tool, receive the tool result, and think again — enabling more helpful and accurate responses.

In [ ]:
graph_with_helpfulness_check = StateGraph(AgentState)

graph_with_helpfulness_check.add_node("agent", call_model)
graph_with_helpfulness_check.add_node("action", tool_node)

Answer:
In this cell, we define `"agent"` as the **entry point** of the graph. This means that whenever the graph execution begins (e.g., when a user query is received), it will start at the `"agent"` node.

The `"agent"` node is responsible for processing the user’s input using the language model and deciding whether to respond directly or call a tool. By setting it as the entry point, we ensure that every interaction starts with reasoning before any action is taken.

In [ ]:
graph_with_helpfulness_check.set_entry_point("agent")

##### YOUR MARKDOWN HERE

Answer:

In the below cell, we define the function tool_call_or_helpful, which acts as a custom routing logic for our LangGraph agent. This function determines what the agent should do next based on the current conversation state.

Here's what each part does:

Tool Call Detection:
If the latest message includes a tool call (tool_calls), the function immediately returns "action", routing the graph to the tool execution node.

Conversation End Check:
If the total number of messages exceeds 10, it returns "END" to stop the conversation and prevent infinite loops.

Helpfulness Evaluation Logic:
If no tool call is present and the message count is acceptable, the function evaluates whether the final agent response is helpful.

It constructs a prompt that compares the initial user query and the agent's final response.

This prompt is passed to a small LLM (gpt-4.1-mini) using a LangChain pipeline composed of:

PromptTemplate

ChatOpenAI

StrOutputParser

Decision Based on Helpfulness:

If the model outputs "Y", the function routes to "end" (i.e., the answer was helpful enough to stop).

If the model outputs "N", it routes to "continue" (i.e., the conversation should continue for refinement).

This logic adds an intelligent layer to the agent, allowing it to self-assess its response and decide whether to continue or end the conversation — making the agent more adaptive and feedback-driven.

In [41]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def tool_call_or_helpful(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  initial_query = state["messages"][0]
  final_response = state["messages"][-1]

  if len(state["messages"]) > 10:
    return END

  prompt_template = """\
  Given an initial query and a final response, determine if the final response is extremely helpful or not. Please indicate helpfulness with a 'Y' and unhelpfulness as an 'N'.

  Initial Query:
  {initial_query}

  Final Response:
  {final_response}"""

  helpfullness_prompt_template = PromptTemplate.from_template(prompt_template)

  helpfulness_check_model = ChatOpenAI(model="gpt-4.1-mini")

  helpfulness_chain = helpfullness_prompt_template | helpfulness_check_model | StrOutputParser()

  helpfulness_response = helpfulness_chain.invoke({"initial_query" : initial_query.content, "final_response" : final_response.content})

  if "Y" in helpfulness_response:
    return "end"
  else:
    return "continue"

#### 🏗️ Activity #4:

Please write what is happening in our `tool_call_or_helpful` function!

Answer:

The tool_call_or_helpful function is responsible for deciding the next step in the agent’s workflow based on the current state of the conversation. Here’s what happens inside the function:
Checks for Tool Calls:
If the latest message from the agent includes a tool call, the function returns "action", which tells the system to execute the requested tool.
Checks for Loop Limit:
If the total number of messages in the conversation exceeds 10, the function returns "END" to stop the process and prevent infinite loops.
Evaluates Helpfulness:
If there is no tool call and the loop limit hasn’t been reached, the function uses a language model to evaluate whether the agent’s latest response is “extremely helpful” for the original user query. It does this by prompting a smaller LLM with both the initial query and the latest response, asking for a “Y” (helpful) or “N” (not helpful).
If the model responds with “Y”, the function returns "end" to finish the conversation.
If the model responds with “N”, it returns "continue", allowing the agent to try again and improve its answer.
In summary:
This function determines whether to use a tool, end the conversation, or let the agent continue, based on tool usage, conversation length, and a helpfulness check using another language model.

##### YOUR MARKDOWN HERE

Answer:

This cell adds conditional routing logic to the graph. After the "agent" node runs, it uses the tool_call_or_helpful function to decide the next step:

If "continue" → loop back to "agent"

If "action" → go to the "action" node to run a tool

If "end" → stop the graph execution

This makes the agent self-evaluate its response and decide whether to act again, try a tool, or finish.

In [42]:
graph_with_helpfulness_check.add_conditional_edges(
    "agent",
    tool_call_or_helpful,
    {
        "continue" : "agent",
        "action" : "action",
        "end" : END
    }
)

##### YOUR MARKDOWN HERE

Answer:
This cell adds an edge from the "action" node back to the "agent" node.
After a tool is used, the agent receives the tool's output and continues reasoning based on it. This creates a loop where the agent can think → act → think again, enabling multi-step reasoning with tools.

In [36]:
graph_with_helpfulness_check.add_edge("action", "agent")

##### YOUR MARKDOWN HERE

Answer:
This cell compiles the graph_with_helpfulness_check into an executable agent called agent_with_helpfulness_check.
It finalizes the structure of the LangGraph so it can be invoked with user inputs. After compilation, the agent is ready to run with the defined nodes, edges, and conditional logic.

In [37]:
agent_with_helpfulness_check = graph_with_helpfulness_check.compile()

##### YOUR MARKDOWN HERE

Answer:

This cell sends a multi-part user query to the agent_with_helpfulness_check and streams its execution step by step.

The input includes three related questions about LoRA, Tim Dettmers, and Attention.

The astream method allows us to observe each update from the graph in real time.

As the agent runs, we see which node is active ("agent" or "action") and the messages being passed.

This helps us understand how the agent thinks, calls tools, and builds its final response across multiple steps.

In [38]:
inputs = {"messages" : [HumanMessage(content="Related to machine learning, what is LoRA? Also, who is Tim Dettmers? Also, what is Attention?")]}

async for chunk in agent_with_helpfulness_check.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xpYdgOQmwga0LOEzu5j7CL3y', 'function': {'arguments': '{"query": "LoRA machine learning"}', 'name': 'wikipedia'}, 'type': 'function'}, {'id': 'call_eTHOsGsPBWrohTxsmQ1Duy8j', 'function': {'arguments': '{"query": "Tim Dettmers"}', 'name': 'wikipedia'}, 'type': 'function'}, {'id': 'call_90ytXeyLg0IDtTMwev3gEHHd', 'function': {'arguments': '{"query": "Attention in machine learning"}', 'name': 'wikipedia'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 265, 'total_tokens': 329, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': None, 'id': 'chatcmpl-BtC9BZKUTtqsg0HeUSx5fhubZor3o', 'service_tier': 'defau

/Users/mayankshah/Documents/ai-makerspace/aie7-bootcamp/AIE7_local/05_Our_First_Agent_with_LangGraph/.venv/lib/python3.13/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/mayankshah/Documents/ai-makerspace/aie7-bootcamp/AIE7_local/05_Our_First_Agent_with_LangGraph/.venv/lib/python3.13/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Receiving update from node: 'action'
[ToolMessage(content='Page: Fine-tuning (deep learning)\nSummary: In deep learning, fine-tuning is an approach to transfer learning in which the parameters of a pre-trained neural network model are trained on new data. Fine-tuning can be done on the entire neural network, or on only a subset of its layers, in which case the layers that are not being fine-tuned are "frozen" (i.e., not changed during backpropagation). A model may also be augmented with "adapters" that consist of far fewer parameters than the original model, and fine-tuned in a parameter-efficient way by tuning the weights of the adapters and leaving the rest of the model\'s weights frozen.\nFor some architectures, such as convolutional neural networks, it is common to keep the earlier layers (those closest to the input layer) frozen, as they capture lower-level features, while later layers often discern high-level features that can be more related to the task that the model is trained

### Task 4: LangGraph for the "Patterns" of GenAI

Let's ask our system about the 4 patterns of Generative AI:

1. Prompt Engineering
2. RAG
3. Fine-tuning
4. Agents

In [39]:
patterns = ["prompt engineering", "RAG", "fine-tuning", "LLM-based agents"]

In [40]:
for pattern in patterns:
  what_is_string = f"What is {pattern} and when did it break onto the scene??"
  inputs = {"messages" : [HumanMessage(content=what_is_string)]}
  messages = agent_with_helpfulness_check.invoke(inputs)
  print(messages["messages"][-1].content)
  print("\n\n")

Prompt engineering is the process of designing and refining prompts to effectively communicate with AI language models, such as GPT, to obtain desired responses. It involves crafting clear, specific, and contextually appropriate prompts to improve the quality, relevance, and accuracy of the AI's outputs. Prompt engineering has become increasingly important as AI models are integrated into various applications, enabling users to leverage their capabilities more effectively.

Prompt engineering started gaining significant attention around 2020-2021, coinciding with the rise of large language models like GPT-3 developed by OpenAI. As these models demonstrated impressive capabilities, the need for effective prompt design to harness their potential became evident, leading to the emergence of prompt engineering as a recognized discipline within AI and NLP communities.





/Users/mayankshah/Documents/ai-makerspace/aie7-bootcamp/AIE7_local/05_Our_First_Agent_with_LangGraph/.venv/lib/python3.13/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/mayankshah/Documents/ai-makerspace/aie7-bootcamp/AIE7_local/05_Our_First_Agent_with_LangGraph/.venv/lib/python3.13/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


RAG can refer to different things, but based on the search results, it appears that "RAG" is not specifically defined in the context of a new technology or trend that "broke onto the scene." However, there is information about "Ragtime," a musical style that was popular from the 1890s to 1910s, characterized by its syncopated rhythm and associated with African American communities.

If you are referring to a different "RAG," such as a recent technological or cultural phenomenon, please provide more context or specify the field you're interested in.



Fine-tuning is a machine learning technique used to adapt a pre-trained model to a specific task or dataset. Instead of training a model from scratch, which can be resource-intensive and time-consuming, fine-tuning involves taking an existing model that has already learned general features from a large dataset and then further training it on a smaller, task-specific dataset. This process helps the model specialize in the new task while le